In [1]:
import pandas as pd

#HUD
#TODO figure out warning 'warn("Workbook contains no default style, apply openpyxl's default")'
df_tz = pd.read_excel("raw_data/hud/TRACT_ZIP_032023.xlsx")
df_zc = pd.read_excel("raw_data/hud/ZIP_COUNTY_032023.xlsx")

#https://gist.github.com/aodin/24c30ba793e404a0270f8c8ef2be350b
df_state = pd.read_csv('raw_data/states_fips.csv', sep=", ", engine='python')

#https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt
df_county = pd.read_csv("raw_data/census/national_county2020.txt", sep="|")
# derive county fips from state and county ids
df_county["COUNTY"] = [int(1000*x + y) for x, y in df_county[["STATEFP", "COUNTYFP"]].values]
df_county = df_county[["COUNTY", "COUNTYNAME", "STATE"]]

# merge tract+zcta and zcta+county data
# focus on residential (drop when 0)
# rename RES_RATIO for tract+zcta and zcta+county data respetively
geog = df_zc.loc[df_zc["RES_RATIO"] != 0, ["ZIP", "COUNTY", "USPS_ZIP_PREF_CITY", "RES_RATIO"]].merge(df_county, on="COUNTY")
geog = geog.rename(columns={"RES_RATIO": "ZC_RATIO"})
geog = geog.merge(df_tz.loc[df_tz["RES_RATIO"] != 0, ["TRACT", "ZIP", "RES_RATIO"]], on="ZIP")
geog = geog.rename(columns={"RES_RATIO": "TZ_RATIO"})

# define tract county ratio (assume homogenity)
geog["TC_RATIO"] = geog["TZ_RATIO"] * geog["ZC_RATIO"]

# normalize column names
geog = geog.rename(columns={"STATE": "stusps"})
geog = geog.merge(df_state, on='stusps')
geog = geog.rename(columns={"st": "STATE", "stname": "STATENAME", "stusps": "STATEABV", "USPS_ZIP_PREF_CITY": "CITYNAME"})

geog = geog[["TRACT", "ZIP", "COUNTY", "STATE",
             "CITYNAME", "COUNTYNAME", "STATENAME", "STATEABV",
             "TZ_RATIO", "ZC_RATIO", "TC_RATIO"]]

# convert to lowercase for easier downstream joining
geog["CITYNAME"] = geog["CITYNAME"].str.lower()
geog["COUNTYNAME"] = geog["COUNTYNAME"].str.lower()
geog["STATENAME"] = geog["STATENAME"].str.lower()

# remove county modifiers
geog["COUNTYNAME"] = geog["COUNTYNAME"].str.replace(" county", "")
geog["COUNTYNAME"] = geog["COUNTYNAME"].str.replace(" municipio", "")
geog["COUNTYNAME"] = geog["COUNTYNAME"].str.replace(" city", "")
geog["COUNTYNAME"] = geog["COUNTYNAME"].str.replace(" census area", "")

geog.to_csv("derived_data/geography.csv", index=False)

C:\Users\hmills1\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\hmills1\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
